In [1]:
%pip install yfinance

In [14]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.optimize as sco
import scipy.stats as st
import statsmodels.api as sm
import plotly.graph_objects as go
import yfinance as yf
import kvantitativne_formule as qt
import pandas_datareader as pdr

First portfolio:
1. Nvidia
2. Ford
3. Boston Scientific Corporation
4. Adobe
5. 3M
6. Abercrombie and Fitch

In [3]:
stocks1 = ["NVDA","F","BSX","ADBE","MMM","ANF"]

Second Porftolio:
1. Visa
2. Rockwell
3. Novartis
4. EA games
5. Mercedes Benz
6. FedEx

In [4]:
stocks2= ["V","ROK","NVS","EA","MBGAF","FDX"]

Downloading the data for the first portfolio

In [5]:
prices1 = yf.download(stocks1, start='2023-01-01',end='2023-09-01')['Adj Close']

[*********************100%%**********************]  6 of 6 completed


Downloading the data for the second one

In [6]:
prices2 = yf.download(stocks2, start='2023-01-01',end='2023-09-01')['Adj Close']

[*********************100%%**********************]  6 of 6 completed


Returns for stock1 data

In [7]:
ret1=prices1.pct_change().dropna()

retunrs for stocks2 data

In [8]:
ret2=prices2.pct_change().dropna()

Efficient frontier for portfolio 1

In [9]:
EF1 = qt.EF(ret1,plot='full')
EF1

Efficient frontier for portfolio 2

In [10]:
EF2 = qt.EF(ret2,plot='full')
EF2

Fine tuning for portfolio 1st EF

In [11]:
EF1_FT = qt.EF(ret1,[0.01,1.5], plot ='full')
EF1_FT

Fine tuning for portfolio 2nd EF

In [12]:
EF2_FT = qt.EF(ret2,[0.01,1.5],'full')
EF2_FT

Taking data from FRED

In [15]:
rfa = pdr.get_data_fred('DTB3',start='2023-1-1',end='2023-9-1')/100
rfa.head()
rfa.dropna()

,DTB3
DATE,
2023-01-03,0.0440
2023-01-04,0.0441
2023-01-05,0.0451
2023-01-06,0.0451
2023-01-09,0.0455
...,...
2023-08-25,0.0534
2023-08-28,0.0534
2023-08-29,0.0533


Risk free data

In [16]:
rf = (1+rfa)**(1/252)-1
rf.head()
rf.dropna()

,DTB3
DATE,
2023-01-03,0.000171
2023-01-04,0.000171
2023-01-05,0.000175
2023-01-06,0.000175
2023-01-09,0.000177
...,...
2023-08-25,0.000206
2023-08-28,0.000206
2023-08-29,0.000206


Max Sharpe's Ratio

In [17]:
opt1 = qt.maxSharpe(ret1,rf)
opt1

{'w': array([ 0.2295113 ,  0.35182633,  0.5268063 , -0.0554469 , -0.69964812,
         0.64695109]),
 'er': 2.1996557829859933,
 'vol': 0.4757153992632251,
 'sharpe': 4.522066678984635}

In [18]:
opt2 = qt.maxSharpe(ret2,rf)
opt2

{'w': array([-0.31152026,  0.59598668, -0.06439689,  0.3511394 , -0.01358148,
         0.44237256]),
 'er': 0.5584193878270312,
 'vol': 0.18336571758149564,
 'sharpe': 2.7812197790052156}

Make plot 1

In [30]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=[qt.pV(opt1['w'],ret1,vol=True)],y=[qt.per(opt1['w'],ret1)],line=dict(color='Blue',width=3),name='Portfolio 1'))
fig.add_trace(go.Scatter(x=[qt.pV(opt2['w'],ret2,vol=True)],y=[qt.per(opt2['w'],ret2)],line=dict(color='Crimson',width=3),name='Portfolio 2'))
fig.update_layout(xaxis=dict(title_text='Risk',zerolinecolor='Black'),yaxis=dict(title_text='Return',zerolinecolor='Black'),
                  title=dict(text="Comparison of two Portfolio's risk/return tradeoffs",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

# <center>Test</center>

Let's test it with investment wealth of 2 million dollars

In [31]:
#Initial wealth
wealth = 2_000_000

In [35]:
#portfolio 1
si1=opt1['w']*wealth

#portfolio2
si2=opt2['w']*wealth

In [37]:
#portfolio1
n1=si1/prices1.iloc[0]

#portfolio2
n2=si2/prices2.iloc[0]

In [38]:
 #portfolio1
 ST1 = prices1@n1

 #portfolio2
ST2 = prices2@n2

Plot

In [41]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=ST1.index,y=ST1,line=dict(color='Blue',width=3),name='Portfolio 1'))
fig.add_trace(go.Scatter(x=ST2.index,y=ST2,line=dict(color='Maroon',width=3),name='Portfolio 2'))
fig.update_layout(xaxis=dict(title_text='Risk',zerolinecolor='Black'),yaxis=dict(title_text='Return',zerolinecolor='Black'),
                  title=dict(text="Comparison of two Portfolio's returns",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

In [44]:
ST3=6.664/2.832
ST3

2.353107344632768

According to trend from the beginning of 2023 untill 1.09.2023, the portfolio 1 which contains stocks from (Nvidia, Adobe, Ford, 3M, BSX and Abercrombie and Fitch) outperform portfolio 2 which contains stocks  from (Visa,Rockwell, Novartis, EA Games, Mercedes Benz and Fedex) 2.35 times. In other words
if you invested at the beginning of the year 2mill in portfolio 1, your portfolio will earn 4.64 mill dolars untill september, while if you invested in the same timespan in portfolio2 you will earn 0.83 mill dollars.